<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>
Rafa Caballero



# Conversión de datos categóricos

### Índice
[Introducción](#Intro)<br>
[Nulos](#Nulos)<br>
[Tipos](#Tipos)<br>
[Variables binarias](#Binarias)<br>
[One-Hot Encoding](#One)<br>
[Labeling](#Labeling)<br>
[Bibliografía](#Bibliografía)<br>

<a name="Intro"></a>
#### Introducción
En muchos algoritmo de inteligencia artificial vamos a necesitar que todos los datos sean numéricos, y más en particular valores sobre los que tenga sentido medir distancias. Por ello una labor importante tras finalizar todos los pasos de limpieza básicos (tratar nulos, duplicados, outliers, hacer giros y agrupar si hace falta, realizar estadísticas básicas, etc.) será convertir las columnas nominales u ordinales (tengan representación numérica o no) a columnas de tipo intervalo/ratio.

El método preferido será el conocido como one-hot-encoding porque preserva las distancias

<img src="https://i.imgur.com/mtimFxh.png" />

Veamos un [ejemplo](https://pbpython.com/categorical-encoding.html):

In [ ]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
cabeceras = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]
df = pd.read_csv(url,header=None, names=cabeceras)
df

<a name="Nulos"></a>
#### Nulos

En este cuardeno no vamos a tratar los nulos en profundidad, solo echamos un vistazo y hacemos lo más sencillo.
Parece que el valor ? significa valor desconocido, vamos a convertirlo en NA. Primero miramos cuántos hay

In [ ]:
(df=='?').sum().sum()

In [ ]:
df2 = df.replace('?',pd.NA)
print(df2.shape)
df2.info()

In [ ]:
df2.isna().sum()

In [ ]:
#pip install missingno

In [ ]:
import seaborn as sns
import missingno as msno
%matplotlib inline

msno.bar(df2)

In [ ]:
msno.matrix(df2)

Como solución sencilla borramos la columna `normalized_loses` y luego las filas que tengan algún nulo (esto no es nada "fino" pero vale para nuestro propósito)

In [ ]:
df3 = df2.drop(columns=["normalized_losses"]).dropna()
msno.matrix(df3)

<a name="Tipos"></a>
#### Tipos

Vamos a ver de qué tipo es cada columna ya que aquí estamos sobre todo interesados en las nominales/ordinales. Pero antes vamos a usar un "truco" para simplicar esta tarea:

- Algunos valores numéricos pueden haberse leído como strings por culpa de los nulos
- Una forma sencilla de convertir es simplemente grabarlo como csv y volver a leer porque así pandas hace la inferencia automática

In [ ]:
from pathlib import Path
path = Path.cwd()
path_procesado = Path(path,"procesado")
path_procesado.mkdir(exist_ok=True)
fichero = Path(path_procesado,"sin_nulos.csv")

df3.to_csv(fichero,index=False)
df3 = pd.read_csv(fichero)

In [ ]:
numericas = df3.select_dtypes("number")
numericas

symboling parece nominal, buscando información sobre esta columna encontramos:  
*A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.*

In [ ]:
df3.symboling.value_counts()

In [ ]:
s = set(df3.columns).difference(set(numericas.columns))
columnas = list(s)

In [ ]:
df3[columnas]

Parece que num_doors y num_cylinders son en realidad tipo ratio, las eliminanos (como curiosidad las vamos a convertir antes)

In [ ]:
!pip install word2number

In [ ]:
from word2number.w2n import word_to_num

word_to_num("three")

In [ ]:
df4 = df3.copy()
df4["num_doors"] = df3.num_doors.map(word_to_num)
df4["num_cylinders"] = df3.num_cylinders.map(word_to_num)
df4["symboling"] = "v"+df3.symboling.astype("str")
df4[["num_doors","num_cylinders","symboling"] ]

<a name="Binarias"></a>
#### Variables binarias

¿Hemos acabado seleccionando variables nominales para transformarlas? Casi, casi, pero aun se puede mirar algo...qué columnas toman solo 2 valores

In [ ]:
numericas = df4.select_dtypes(include=["number"])
s = set(df4.columns).difference(set(numericas))
columnas = list(s)
for c in columnas:
    print(c,len(df4[c].unique()))


Las variables que tiene solo 2 valores no necesitan one-hot-encoding, las podemos transformar directamente en 0 y 1


In [ ]:
df5 = df4.copy()
transformados = []
for c in columnas:
    valores = df4[c].unique()
    if len(valores) == 2:
        # vamos a construir un diccionario con los dos valores a 0 y a 1
        df5[c] = df4[c].replace({valores[0]:0, valores[1]:1})
        transformados.append([c,valores[0],valores[1]])
df5

Resulta útil guardar los valores transformados por si luego hay que hacer la transformación inversa

In [ ]:
df_transformados = pd.DataFrame(data=transformados,columns=["columna","valor0","valor1"])
df_transformados

In [ ]:
file_binarias = Path(path_procesado,"binarias.csv")
df_transformados.to_csv(file_binarias,index=False)

Usamos el mismo truco de grabar y leer para recuperar los tipos


In [ ]:

fichero = Path(path_procesado,"con_binarias.csv")

df5.to_csv(fichero,index=False)
df5 = pd.read_csv(fichero)
df5

<a name="One"></a>
#### One-Hot Encoding

¡Por fin! Vamos a aplicar el método one-hot-encoding

In [ ]:
nominales = df5.select_dtypes(exclude='bool').select_dtypes(exclude='number')
nominales

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore',sparse=False)  # 1 método
enc.fit(nominales)  #  2 el método "aprende"
salida = enc.transform(nominales) # 3 aplicamos el método
salida

Afortunadamente es fácil convertir esta matriz en un dataframe

In [ ]:
encoded_df = pd.DataFrame(
     salida,
     columns=enc.get_feature_names_out()
)
encoded_df

Ahora juntamos todo: por un lado el dataframe restringido a las columnas que no se han visto afectadas por el one-hot-encoding y por otro el resultado del one-hot-encoding

In [ ]:
resto_cs = list(set(df5.columns).difference(set(nominales.columns)))
final = pd.concat([df5[resto_cs],encoded_df],axis=1)
final

Nota sobre **correlaciones y variables nominales**:


 En el caso de variables nominales es muy habitual la secuencia:
1) Buscar asociaciones con  $\mathcal{X}^2$ entre variables nominales, nos dice si hay alguna asociación pero no entre qué valores
2) Convertir a variables dummy con one-hot-encoding
3) Repetir las correlaciones, ahora ya como variables numéricas con `corr()`que nos mostrará posibles correlaciones con otras variables ratio y entre sí, pero ahora entre valores individuales de la variable nominal y distinguiendo entre correlación positiva y negativa


<a name="Labeling"></a>
#### Labeling

Hay veces que no hace falta complicarse tanto.


Por ejemplo si es una variable a prededecir (la y):
    1.- no queremos transformarla en varias
    2.- No se van a calcular distancias con ellas
    

Otra caso es cuando el método a utilizar no es de naturaleza geométrica (árboles de decisión por ejemplo).

En este caso podemos usar un label encoder. Por ejemplo supongamos que quiero codificar la columna `make`

In [ ]:
df5

In [ ]:
from sklearn.preprocessing import LabelEncoder

df6 = df5.copy()

enc = LabelEncoder()  # 1 método
enc.fit(df5.make)  #  2 el método "aprende"
df6["make"] = enc.transform(df5.make) # 3 aplicamos el método
df6


Además tenemos la inversa a mano en cualquier momento:

In [ ]:
enc.inverse_transform(df6.make)

In [ ]:
enc.inverse_transform(list(range(21)))


<a name="Bibliografía"></a>
#### Bibliografía

[Otro tipo de encoders más complejos](https://pbpython.com/categorical-encoding.html)

[Comparativa de one-hot encoding y otro método get_dummies](https://pythonsimplified.com/difference-between-onehotencoder-and-get_dummies/)